## Tables
* Assume that [set4delta.csv] file is located in the same directory.
* "datacall.r" file preprocess the [set4delta.csv] dataset. (source("datacall.r") would take few minutes)
* functions for replicating the results in simulation studies are coded in "simulation.r".

In [15]:
myfile<-read.csv("set4delta.csv")
lfc = myfile$realdata

In [17]:
source("datacall.r")
source("simulation.r")

Warning message in locfdr(trndat, plot = 0, type = 0, bre = 1000):
“f(z) misfit = 1.6.  Rerun with increased df”


        label4     realdata          WT1          WT2          WT3          KO1
1         ICR1  0.002630448 3.646573e+03 4.019110e+03 4.137667e+03 3.793537e+03
2         IRT1 -0.061235143 1.760300e+03 2.013854e+03 2.086823e+03 1.871974e+03
3         LSR1  0.011810776 1.949654e+06 2.401455e+06 2.193183e+06 2.143502e+06
4   MSTRG.10.1  0.327721125 1.823287e+01 4.776694e+00 4.831562e+01 3.146976e+01
5 MSTRG.1001.1  0.023151009 5.005750e+02 4.174830e+02 4.533444e+02 4.520202e+02
6 MSTRG.1006.1  0.088136981 6.688147e+02 6.563177e+02 7.072584e+02 7.180827e+02
           KO2          KO3     realcv       cvKO        cvWT     Max.CV
1 3.970038e+03 4.061316e+03 0.06513217 0.06513217 0.034536752 0.06513217
2 1.751374e+03 1.994066e+03 0.08772375 0.08772375 0.064805792 0.08772375
3 2.296157e+06 2.158429e+06 0.10366110 0.10366110 0.038264590 0.10366110
4 3.186072e+01 2.618471e+01 0.93763360 0.93763360 0.106246226 0.93763360
5 5.078405e+02 4.337262e+02 0.09116674 0.09116674 0.083112324 0.09116674
6 

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, table,
    tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading r

#### Simulation for **Table 1** 
* with sample size(n) 8000
* differentiating the correlations of primary p-values and auxiliary p-values by [0.0, 0.2, 0.4, 0.6, 0.8]
* differentiating the distance between null($N(0,1)$) and alternative($N(mm, 1)$) distributions.
* This will take few hours.

In [24]:
n = 8000
taus = c(0, 0.2, 0.4, 0.6, 0.8)
mms = c(2.0, 2.5, 3.0, 3.5, 4.0)

In [ ]:
for(tau in c(0, 0.2,0.4, 0.6, 0.8)){
  print(paste(c("now the tau is", tau)))
  
  simres = array(0.0, c(2, 21, 1000) )
  if(tau==0){
    cop = BiCop(0)
  }else{
    cop = BiCop(3, tau = tau)
  }
  
  for(i in 1:1000){
    simdat = simgen(i, n, cop)
    res = test_all(simdat)
    simres[,,i] = res
    if(i%%10==0) print(res[1,1:21])
  }
  assign(paste0("tau_",tau), simres)
}

In [ ]:
for(mm in c(2.0, 2.5, 3.0, 3.5, 4.0)){
  print(paste(c("now the mu is", mm)))
  
  simres = array(0.0, c(2, 21, 1000))
  if(tau==0){
    cop = BiCop(0)
  }else{
    cop = BiCop(3, tau = 0.4)
  }
  
  for(i in 1:1000){
    simdat = simgen(i, n, cop, mm=mm)
    res = test_all(simdat)
    simres[,,i] = res
    if(i%%100==0) print(res[1,1:21])
  }
  assign(paste0("mm_",mm), simres)
  #saveRDS(simres, paste0("cop23_tau",tau,"_mm2_sd1.rds"))
}

In [ ]:
fdr_tbl = array(NA, dim = c(7,4,5))
power_tbl = fdr_tbl

i = 1

for(tau in c(0,0.2,0.4,0.6,0.8)){
  mat = get(paste0("tau_",tau))
  fdr_tbl[,,i] = fdr_and_power(mat)$fdr
  power_tbl[,,i] = fdr_and_power(mat)$power
  i = i+1
}

saveRDS(fdr_tbl, "fdr_diff_tau.rds")
saveRDS(power_tbl, "power_diff_tau.rds")

fdr_tbl2 = array(NA, dim = c(7,4,5))
power_tbl2 = fdr_tbl

i = 1

for(mu in c(2, 2.5, 3, 3.5, 4)){
  mat = get(paste0("mm_",mu))
  fdr_tbl2[,,i] = fdr_and_power(mat)$fdr
  power_tbl2[,,i] = fdr_and_power(mat)$power
  i = i+1
}

saveRDS(fdr_tbl2, "fdr_diff_mu.rds")
saveRDS(power_tbl2, "power_diff_mu.rds")

#### Simulation for **Table 2** 
* with sample size(n) 8000
* the correlations of primary p-values and auxiliary p-values is set to be 0.4
* the distance between null($N(0,1)$) and alternative($N(3, 1)$) distributions.
* compared with Misspecified copula : BB7, BB6 and Joe copulas
* This will take few hours.

In [ ]:
miscop = c(39, 38,36) #BB7, BB6, Joe
for(c in miscop){
  print(paste(c("now the cop is", c)))
  cop = BiCop(3, tau = 0.4)
  simres = array(0.0, c(2, 21, 1000) )
  for(i in 1:1000){
    simdat = simgen(i, n, cop)
    res = test_all(simdat, copnum = c)
    simres[,,i] = res
    if(i%%100==0) print(res[1,1:21])
  }
  assign(paste0("misfit_",c), simres)
}

In [ ]:
miscop = c(39, 38,36) #BB7, BB6, Joe
for(c in miscop){
  print(paste(c("now the cop is", c)))
  testcop = BiCop(3, tau = 0.4)
  simres = array(0.0, c(2, 21, 1000))
  for(i in 1:1000){
    set.seed(i)
    rr = BiCopSim(n, obj = testcop)
    cop = BiCopEst(rr[,1], rr[,2], c)
    simdat = simgen(i, n, cop)
    res = test_all(simdat)
    simres[,,i] = res
    if(i%%100==0) print(res[1,1:21])
  }
  assign(paste0("misgen_",c), simres)
}

In [ ]:
fdr_tbl3 = array(NA, dim = c(7,4,5))
power_tbl3 = fdr_tbl

i = 1

for(c in miscop){
  mat = get(paste0("misfit_",c))
  fdr_tbl3[,,i] = fdr_and_power(mat)$fdr
  power_tbl3[,,i] = fdr_and_power(mat)$power
  i = i+1
}

saveRDS(fdr_tbl3, "fdr_misfit.rds")
saveRDS(power_tbl3, "power_misfit.rds")





fdr_tbl4 = array(NA, dim = c(7,4,5))
power_tbl4 = fdr_tbl

i = 1

for(c in miscop){
  mat = get(paste0("misgen_",c))
  fdr_tbl4[,,i] = fdr_and_power(mat)$fdr
  power_tbl4[,,i] = fdr_and_power(mat)$power
  i = i+1
}

saveRDS(fdr_tbl4, "fdr_misgen.rds")
saveRDS(power_tbl4, "power_misgen.rds")

#### **Table 3** The Log-likelihood, AIC, BIC of Gaussian, Frank, Clayton, Gumbel copulas with set4∆ dataset

In [18]:
tau = wdm(pval1, pval2, "kendall"); tau
cop = BiCop(23, tau = tau)

[1] -0.3862325

In [30]:
pval11 = pval1[pval1 + pval2 >= 1]
pval22 = pval2[pval1 + pval2 >= 1]
pval111 = c(pval11, 1-pval22)
pval222 = c(pval22, 1-pval11)

In [36]:
aa = BiCopEstList(pval11, pval22, c(1,5,23,24,26), rotations = F)$summary
aa$family = c("Gaussian", "Frank", "Clayton","Gumbel", "Joe"); aa

family,logLik,AIC,BIC
<chr>,<dbl>,<dbl>,<dbl>
Gaussian,306.45,-610.91,-604.80
Frank,332.84,-663.68,-657.57
Clayton,393.08,-784.16,-778.05
Gumbel,201.79,-401.57,-395.46
Joe,96.29,-190.57,-184.46


#### **Table4** Comparison of total and HXT family gene rejections using one-stage (locfdr, Storey), covarate-assisted (IHW, Boca and Leek, AdaFDR) and two-stage (types H and S) FDR methods at α = 0.05 and α = 0.10.

In [281]:
## locfdr
rej_locfdr = (p0hat*f0/f < 0.05)
rej_locfdr2 = (p0hat*f0/f < 0.1)


## Storey
p0 = pi_01(pval2)
rej_Storey = (p0*n*pval2/rank(pval2) <0.05)
rej_Storey2 = (p0*n*pval2/rank(pval2) <0.1)

## Two-stage (H)
gamma_seq = seq(0.5,1,0.001)
tau = wdm(pval1, pval2, "kendall")
cop = BiCop(23, tau = tau)

nrej = sapply(gamma_seq, function(x){
  pv = BiCopCDF(rep(x, n), pval2, obj = cop)
  pv = ifelse(pval1>x, pval1, pv)
  p0 = pi_01(pv)
  nr =  sum(p0*n*pv/rank(pv) <0.10)
  return(nr)
})


gamma = gamma_seq[which.max(nrej)]; gamma

pv = BiCopCDF(rep(gamma, n), pval2, obj = cop)
pval_H = ifelse(pval1>gamma, pval1, pv)
p0 = pi_01(pval_H)
rej_H = (p0*n*pval_H/rank(pval_H) <0.05)
rej_H2 = (p0*n*pval_H/rank(pval_H) <0.10)

## Two-stage (S)
pval_S = BiCopHfunc1(pval1, pval2, obj = cop)
p0hat = pi_01(pval_S)
rej_S2 = (p0hat*n*pval_S/rank(pval_S) <0.10)
rej_S = (p0hat*n*pval_S/rank(pval_S) <0.05)

## IHW
res_ihw <- ihw(pval2 ~ pval1, alpha = 0.05)
pval_ihw = adj_pvalues(res_ihw)
  
rej_ihw = pval_ihw < 0.05
rej_ihw2 = pval_ihw < 0.10

## BL
qvalues <- lm_qvalue(pval2, X=pval1)
newp = qvalues$qvalues
test = p.adjust(newp,"BH")
  
rej_BL = test < 0.05
rej_BL2 = test < 0.10


## adaFDR
rej_ada <- adafdr_test(pval2, pval1, alpha = 0.05)$decision
rej_ada2 <- adafdr_test(pval2, pval1, alpha = 0.10)$decision

[1] 0.987

In [294]:
rej1 = data.frame(locfdr = rej_locfdr, storey = rej_Storey, ihw = rej_ihw, BL = rej_BL, adaFDR = rej_ada, typeH = rej_H, typeS = rej_S)
rej2 = data.frame(locfdr = rej_locfdr2, storey = rej_Storey2, ihw = rej_ihw2, BL = rej_BL2, adaFDR = rej_ada2, typeH = rej_H2, typeS = rej_S2)
apply(rej1, 2, sum) # # of rejected genes with alpha = 0.05
apply(rej2, 2, sum) # # of rejected genes with alpha = 0.10
apply(rej1, 2, function(x) sum(x[hxtloc])) # # of rejected HXTfamily with alpha = 0.05
apply(rej2, 2, function(x) sum(x[hxtloc])) # # of rejected HXTfamily with alpha = 0.10

locfdr storey    ihw     BL adaFDR  typeH  typeS 
   230    361    486    200    844    407    503

locfdr storey    ihw     BL adaFDR  typeH  typeS 
   249    424    633    228   1329    485    586

locfdr storey    ihw     BL adaFDR  typeH  typeS 
     3      4      5      3      6      3      4

locfdr storey    ihw     BL adaFDR  typeH  typeS 
     3      4      5      3      6      4      6

#### Table S1: The number of selected copula based on LogLik, AIC and BIC when generating random variables using the Clayton copula with mean and standard deviation.

In [20]:
fam = c(1,5,23,24,26)
names(fam) = c("Gaussian", "Frank", "Clayton","Gumbel","Joe")
cop = BiCop(23, tau = -0.4)

In [21]:
lik = matrix(0,100,length(fam))
aic = matrix(0,100,length(fam))
bic = matrix(0,100,length(fam))

for(i in 1:100){
    set.seed(i)
    r = BiCopSim(8000, obj = cop)
  for(f in 1:length(fam)){
    estcop = BiCopEst(r[,1], r[,2], fam[f])
    lik[i,f] = estcop$logLik
    aic[i,f] = estcop$AIC
    bic[i,f] = estcop$BIC 
  }
}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100


In [27]:
t(round(apply(lik, 2, function(x) c(mean(x), sd(x))),3))
t(round(apply(aic, 2, function(x) c(mean(x), sd(x))),3))
t(round(apply(bic, 2, function(x) c(mean(x), sd(x))),3))

1615.157,57.301
1524.409,53.864
2194.153,71.345
1090.186,50.598
501.012,35.078
2187.548,70.794
2184.828,71.031


-3228.315,114.602
-3046.818,107.729
-4386.307,142.690
-2178.372,101.197
-1000.024,70.156
-4371.095,141.589
-4365.656,142.062


-3221.328,114.602
-3039.830,107.729
-4379.320,142.690
-2171.385,101.197
-993.037,70.156
-4357.121,141.589
-4351.682,142.062


In [25]:
table(apply(lik, 1, which.max))
table(apply(aic, 1, which.min))
table(apply(bic, 1, which.min))


 3  6 
94  6 


 3  6 
96  4 


 3  6 
99  1 

#### **Table S2** The Log-likelihood, AIC, BIC of Gaussian, Frank, Clayton, Gumbel, BB6, BB7 and Joe copulas with set4∆ dataset

In [37]:
aa = BiCopEstList(pval1_deseq, pval2, family = c(1,5,23,24,26), rotations = F)$summary
aa$family = c("Gaussian", "Frank", "Clayton","Gumbel", "Joe")
aa

family,logLik,AIC,BIC
<chr>,<dbl>,<dbl>,<dbl>
Gaussian,1578.79,-3155.57,-3148.56
Frank,1486.99,-2971.97,-2964.96
Clayton,836.13,-1670.25,-1663.24
Gumbel,1199.29,-2396.58,-2389.56
Joe,486.19,-970.38,-963.37
